In [1]:
#Import the Dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy
from datetime import datetime
import requests
# Import the API key.
from config import weather_api_key

In [2]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [3]:
#Create a set of random latitude and longitude combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)

In [4]:
coordinates = list(lat_lngs)

In [5]:
#Create a list for holding the cities
cities = []
#Identify the nearest city for each latitude and longitude combination
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    #If the city is unique, then we will add it to the cities list
    if city not in cities:
        cities.append(city)
#print the city count to confirm sufficient count
len(cities)

734

In [6]:
#Create an empty list to hold the the weather data
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval.       ")
print("--------------------------------")

#Create counters.
record_count = 1
set_count = 1

#Loop through all the cities in our list.
for i, city in enumerate(cities):
    
    # Group cities in sets of 50 for logging purposes
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ", "+")
    
    # Log the URL, record, and set unbers and the city
    #print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count +=1
    
    #Run and API request for each of the cities.
    try:
        #Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        #Parse the needed data
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        current_description = city_weather['weather'][0]['description']
        #Convert the date to ISO standard
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Current Description": current_description,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        #print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")


Beginning Data Retrieval.       
--------------------------------
-----------------------------
Data Retrieval Complete      
-----------------------------


In [7]:
len(city_data)

670

In [8]:
#Convert the array of dictionaries to a Pandas dataframe
city_data_df = pd.DataFrame(city_data)
#city_data_df.head(10)

In [9]:
new_column_order = ["City", "Country", "Date", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed","Current Description"]
city_data_df = city_data_df[new_column_order]
city_data_df.head(10)

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Mataura,NZ,2021-04-23 20:51:01,-46.1927,168.8643,45.00,95,53,1.99,broken clouds
1,Khatanga,RU,2021-04-23 20:51:01,71.9667,102.5000,33.06,97,100,21.21,overcast clouds
2,Tiksi,RU,2021-04-23 20:51:01,71.6872,128.8694,4.64,98,97,7.18,overcast clouds
3,Maputo,MZ,2021-04-23 20:51:02,-25.9653,32.5892,64.00,88,75,4.61,broken clouds
4,Kodiak,US,2021-04-23 20:51:02,57.7900,-152.4072,39.20,87,90,9.22,light rain
5,Tasiilaq,GL,2021-04-23 20:51:02,65.6145,-37.6368,39.20,45,90,5.75,overcast clouds
6,Vostok,RU,2021-04-23 20:51:02,46.4856,135.8833,40.96,85,100,6.40,overcast clouds
7,Te Anau,NZ,2021-04-23 20:51:03,-45.4167,167.7167,45.64,87,64,2.62,light rain
8,Saskylakh,RU,2021-04-23 20:51:03,71.9167,114.0833,33.58,96,100,18.95,light snow
9,Jamestown,US,2021-04-23 20:49:29,42.0970,-79.2353,53.60,32,1,16.11,clear sky


In [13]:
# Create the output file (CSV)
output_data_file = "Weather_Database/WeatherPy_Database.csv"
# Export the dataframe to a csv
city_data_df.to_csv(output_data_file, index_label="City_ID")